# Configurações iniciais

In [1]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zipgit 
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import numpy as np
import pandas as pd  # Manipulação de dados



# Configurações das bibliotecas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

# Extração dos dados

In [2]:
# Extrair o arquivo excel da pasta ZIP diretamente do site de download
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 5 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3  42 non-null     object
 1   Unnamed: 1                                            58 non-null     object
 2   Unnamed: 2                                            530 non-null    object
 3   Unnamed: 3                                            458 non-null    object
 4   Unnamed: 4                                            285 non-null    object
dtypes: object(5)
memory usage: 22.7+ KB


In [4]:
df.memory_usage()

Index                                                    132
CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3    4624
Unnamed: 1                                              4624
Unnamed: 2                                              4624
Unnamed: 3                                              4624
Unnamed: 4                                              4624
dtype: int64

# Transformação dos dados

In [5]:
df.columns = df.iloc[5].values
df = df.iloc[5:]
df = df.rename(columns={'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', np.nan: 'LISTAGEM'})[1:]
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']
df = df.dropna(how='all')
df = df[['CÓDIGO', 'NOME', 'SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'LISTAGEM']]
df['SETOR ECONÔMICO'].ffill(inplace=True)
df['SUBSETOR'].ffill(inplace=True)
df['SEGMENTO'].ffill(inplace=True)
df.LISTAGEM.fillna(value='Sem classificação', inplace=True)
df = df.apply(lambda x: x.astype('category') if (x.dtype == 'O') and (x.name not in ['CÓDIGO', 'NOME']) else x)
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]
df = df.reset_index(drop=True)
df.to_parquet(path='./tickers_por_setor.parquet')

In [6]:
display(df)

,CÓDIGO,NOME,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM
0,RRRP,3R PETROLEUM,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
1,CSAN,COSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,ENAT,ENAUTA PART,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
3,RPMG,PET MANGUINH,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",Sem classificação
4,PETR,PETROBRAS,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
...,...,...,...,...,...,...
427,PPAR,POLPAR,Outros,Outros,Outros,Sem classificação
428,PRPT,PROMPT PART,Outros,Outros,Outros,Sem classificação
429,OPSE,SUDESTE S/A,Outros,Outros,Outros,Sem classificação
430,OPTS,SUL 116 PART,Outros,Outros,Outros,Sem classificação


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CÓDIGO           432 non-null    object  
 1   NOME             432 non-null    object  
 2   SETOR ECONÔMICO  432 non-null    category
 3   SUBSETOR         432 non-null    category
 4   SEGMENTO         432 non-null    category
 5   LISTAGEM         432 non-null    category
dtypes: category(4), object(2)
memory usage: 14.6+ KB


In [8]:
df.memory_usage()

Index               132
CÓDIGO             3456
NOME               3456
SETOR ECONÔMICO    1728
SUBSETOR           1864
SEGMENTO           3200
LISTAGEM           1108
dtype: int64